## PROCESO ETL

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import ast

####  Datasets

In [8]:
#Ingresamos link del Dataset Movies
url_movies = "https://drive.google.com/uc?export=download&id=1Rp7SNuoRnmdoQMa5LWXuK4i7W1ILblYb"

In [12]:
#Ingresamos Ruta del Dataset Creditos
creditos = "creditos.parquet"

#### dataframes

In [10]:
#dataframe movies
df = pd.read_csv(url_movies)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_15120\879329235.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url_movies)


In [13]:
#dataframe creditos
dfc = pd.read_parquet(creditos)

#### Transformaciones

In [12]:
# Eliminamos columnas que no necesitamos
df.drop(columns=['video','imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [64]:
# valores nulos
df.isnull().sum()

budget                      0
genres                      0
id                          0
original_language          11
overview                  954
popularity                  5
production_companies        3
production_countries        3
release_date               87
revenue                     0
runtime                   263
spoken_languages            6
status                     87
tagline                 25054
title                       6
vote_average                6
vote_count                  6
dtype: int64

In [16]:
#Imprimir tipo de dato de cada atributo
df.dtypes

belongs_to_collection     object
budget                    object
genres                    object
id                        object
original_language         object
overview                  object
popularity                object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
vote_average             float64
vote_count               float64
dtype: object

#### Reemplazamos los valores nulos de Revenue y Budget por 0

In [17]:
df["revenue"] = df["revenue"].fillna(0)
df["budget"] = df["budget"].fillna(0)

In [18]:
#Comprobar nuevamente los valores nulos
df.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

### Desanidar el atributo belongs_to_collection

In [22]:
df['belongs_to_collection'].head(5)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [23]:
#Definir un diccionario por defecto para reemplazar los valores NaN
default_dict = {'id': np.nan, 'name': np.nan, 'poster_path': np.nan, 'backdrop_path': np.nan}

def replace_nan_with_dict(x):
    if pd.isna(x):
        return default_dict
    return x

def safe_literal_eval(x):
    """Evaluates a string as a Python literal if possible, otherwise returns the original value."""
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

# Convertir las cadenas en literales y reemplazamos NaN con el diccionario predeterminado
df['belongs_to_collection'] = df['belongs_to_collection'].apply(safe_literal_eval).apply(replace_nan_with_dict)

# Separar los componentes del diccionario en columnas nuevas
df2 = df['belongs_to_collection'].apply(pd.Series)

print(df2.columns)

Index(['id', 'name', 'poster_path', 'backdrop_path', 0], dtype='object')


Se considera que puede ser relevante solamente el nuevo atributo 'name' y es por ello que eliminamos aquellos no relevantes

In [24]:
df2.drop(columns=[0, 'id',  'poster_path', 'backdrop_path'], inplace=True)
#Renombrar la nueva columna 'name'
df2 = df2.rename(columns = {'name' : 'name_coll'})

In [25]:
#Comprobar cantidad de valores nulos en la columna generada
df2['name_coll'].isnull().sum()

40975

In [26]:
# Eliminar la columna original 'belongs_to_collection'
df.drop('belongs_to_collection', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
df3bc = pd.concat([df, df2], axis=1)

#### Desanidar el atributo genres

In [28]:
#Inspeccionar la estructura del tipo de dato del atributo 'genres'
df.genres.loc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [29]:
# Evaluar los strings como listas de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Expandir los diccionarios en columnas
df2g = df['genres'].explode().apply(pd.Series)

In [31]:
#Imprimir el nombre de las columnas
df2g.columns

Index(['id', 'name', 0], dtype='object')

Se considera que puede ser relevante solamente el nuevo atributo 'name' y es por ello que eliminamos aquellos no relevantes

In [32]:
#Eliminar la columna 0 y 'id'
df2g.drop(columns=[0, 'id'], inplace=True)

In [33]:
#Imprimir cantidad de valores nulos
df2g.isnull().sum()

name    2442
dtype: int64

In [34]:
#Agrupar las filas por el índice original de df y concatenar los valores
# de la columna 'name' en una cadena separada por comas.
df2g_ind = df2g.groupby(df2g.index)['name'].apply(lambda x: ','.join(x.astype(str))).reset_index()

In [36]:
#Comprobar que la cantidad de valores nan de tipo string se mantenga
df2g_ind['name'].value_counts().get('nan', 0)

2442

In [37]:
#Renombrar la nueva columna 'name'
df2gg = df2g_ind.rename(columns = {'name' : 'name_gen'})


In [38]:
#Extraer la columna de interés
df2gg = df2gg[['name_gen']]


In [39]:
# Eliminar la columna original 'genres' del dataframe usado en el item anterior
df3bc.drop('genres', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
df3g = pd.concat([df3bc, df2gg], axis=1)

#### Desanidar el atributo 'production_companies'

In [41]:
#Inspeccionar la estructura del tipo de dato del atributo 'production_companies'
df['production_companies'][:2]

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
Name: production_companies, dtype: object

In [42]:
df.production_companies.loc[1]

"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"

In [43]:
df.production_companies.loc[50]

'[]'

In [44]:
df.production_companies.loc[6]

"[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Scott Rudin Productions', 'id': 258}, {'name': 'Mirage Enterprises', 'id': 932}, {'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Constellation Entertainment', 'id': 14941}, {'name': 'Worldwide', 'id': 55873}, {'name': 'Mont Blanc Entertainment GmbH', 'id': 58079}]"

Se observa que los elementos del atributo 'production_companies' están formados por listas de diccionarios y listas vacías. Extraemos solamente los nombres de las productoras ya que podrían ser útiles

In [45]:
# Función para extraer los nombres
def extraer_nombres(diccionarios):
    if isinstance(diccionarios, str):
        try:
            diccionarios = ast.literal_eval(diccionarios)
        except (SyntaxError, ValueError):
            return np.nan

    if not isinstance(diccionarios, list):
        return np.nan

    nombres = [diccionario.get('name', np.nan) for diccionario in diccionarios if diccionario.get('name') is not None]

    if not nombres:
        return np.nan

    return ', '.join(nombres)

# Aplicar la función a la columna 'production_companies'
df2pc = df['production_companies'].apply(extraer_nombres)

In [46]:
df2pc[:2]

0                              Pixar Animation Studios
1    TriStar Pictures, Teitler Film, Interscope Com...
Name: production_companies, dtype: object

In [47]:
# Crear DataFrame con la serie df2pc
df22 = pd.DataFrame(df2pc.tolist(), columns=['production_companies'])

In [48]:
df22[:2]

,production_companies
0,Pixar Animation Studios
1,"TriStar Pictures, Teitler Film, Interscope Com..."


In [49]:
# Eliminar la columna original 'production_companies' del dataframe usado en el item anterior
df3g.drop('production_companies', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
df3_pcomp = pd.concat([df3g, df22], axis=1)

In [50]:
df3_pcomp[:2]

,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,name_coll,name_gen,production_companies
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"Adventure,Fantasy,Family","TriStar Pictures, Teitler Film, Interscope Com..."


#### Desanidar el atributo production_countries

In [51]:
#Inspeccionar el tipo de dato de 'production_countries'
df['production_countries'][:2]

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

In [52]:
df2pcou = df['production_countries'].apply(extraer_nombres)

In [53]:
df2pcou[:2]

0    United States of America
1    United States of America
Name: production_countries, dtype: object

In [54]:
# Crear DataFrame con la serie df2pcou
df22pcou = pd.DataFrame(df2pcou.tolist(), columns=['production_countries'])

In [55]:
df22pcou[:2]

,production_countries
0,United States of America
1,United States of America


In [56]:
# Eliminar la columna original 'production_companies' del dataframe usado en el item anterior
df3_pcomp.drop('production_countries', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
df33_pcomp = pd.concat([df3_pcomp, df22pcou], axis=1)

In [57]:
df33_pcomp[:2]

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,name_coll,name_gen,production_companies,production_countries
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"Adventure,Fantasy,Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America


#### Atributo 'spoken_languages'

In [58]:
#Inspeccionar el tipo de dato
df.spoken_languages.iloc[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

In [59]:
df2lang = df['spoken_languages'].apply(extraer_nombres)

In [60]:
df2lang[:2]

0              English
1    English, Français
Name: spoken_languages, dtype: object

In [61]:
# Crear DataFrame con la serie df2lang
df22lang = pd.DataFrame(df2lang.tolist(), columns=['languages'])

In [62]:
df22lang[:2]

,languages
0,English
1,"English, Français"


In [63]:
# Eliminar la columna original 'spoken_languages' del dataframe usado en el item anterior
df33_pcomp.drop('spoken_languages', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
df3 = pd.concat([df33_pcomp, df22lang], axis=1)

#### Atributo 'revenue' y 'budget'

In [65]:
#Inspeccionar el tipo de dato de cada atributo
print(df3['revenue'].dtypes)
print(df3['budget'].dtypes)

float64
object


Dado que el atributo "budget" es de tipo "object",se revisan sus valores y luego se convierten a tipo numérico

In [66]:
# Función para comprobar si una cadena contiene punto o coma
def contains_punctuations(value):
    if isinstance(value, str):
        return bool(re.search(r'[.,]', value))
    return False

In [67]:
# Aplicar la función a la columna 'budget' y obtenemos los índices que cumplen la condición
indices = df3[df3['budget'].apply(contains_punctuations)].index

# Mostrar los índices
print("Indices:", indices.tolist())

# Mostrar los elementos para los índices correspondientes
elementos = df3.loc[indices]

Indices: [19730, 29503, 35587]


Reemplazar los strings encontrados por 0

In [68]:
# Índices específicos a reemplazar
indices_reempl = [19730, 29503, 35587]

# Reemplazar los valores en los índices específicos por 0
for idx in indices_reempl:
    if idx in df3.index:
        df3.at[idx, 'budget'] = 0

In [69]:
# Verificar los cambios
print(df3.loc[indices_reempl]['budget'])

19730    0
29503    0
35587    0
Name: budget, dtype: object


In [70]:
#Convertir la variable a tipo numérico
df3['budget'] = pd.to_numeric(df3['budget'], errors='coerce')


Se realiza el cálculo del **retorno de inversión**

In [71]:
df3['return'] = df3.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)


In [72]:
df3['return'][:5]

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

#### Atributo 'popularity'

Inspeccionar el tipo de dato de 'Popularity'

In [73]:
df3['popularity'][:5]

0    21.946943
1    17.015539
2      11.7129
3     3.859495
4     8.387519
Name: popularity, dtype: object

In [74]:
#Conviertir la columna a tipo numérico
df3['popularity'] = pd.to_numeric(df3['popularity'], errors='coerce')

In [75]:
#Comprobar el cambio
df3['popularity'][:5]

0    21.946943
1    17.015539
2    11.712900
3     3.859495
4     8.387519
Name: popularity, dtype: float64

#### Atributo 'realease_date' y nuevo atributo 'rel_year'

Inspeccionar el tipo de dato de 'realease_date'

In [76]:
df3['release_date'][:5]

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [78]:
#Contar cantidad de datos nulos
df3['release_date'].isna().sum()

87

In [79]:
#Convertir la variable de tipo object a tipo fecha
#Utilizar errors='coerce' para manejar los valores NaN
df3['release_date'] = pd.to_datetime(df3['release_date'], errors='coerce' )

In [80]:
#Eliminar valores nulos
df3 = df3.dropna(subset=['release_date'])

In [81]:
#Crear nuevo atributo que contiene año de estreno de la película
df3['release_year'] = df3['release_date'].dt.year.astype('int32')

In [82]:
df3['release_year'][:2]

0    1995
1    1995
Name: release_year, dtype: int32

#### Duplicados

In [83]:
# Contar cantidad de duplicados en la columna 'id'
dupl1 = df3['id'].duplicated(keep=False)
print(dupl1.sum())

59


In [84]:
# Mostrar duplicados por la columna 'id'
dupl11 = df3[df3.duplicated(subset=['id'], keep=False)]

In [85]:
# Contar la frecuencia de cada 'id'
frecuencia_id = df3['id'].value_counts()

In [86]:
# Filtrar solo los 'id' que se repiten
id_repetidos = frecuencia_id[frecuencia_id > 1]

print("\n'id' repetidos:")
print(id_repetidos)


'id' repetidos:
id
141971    3
97995     2
10991     2
109962    2
119916    2
159849    2
84198     2
132641    2
168538    2
99080     2
18440     2
12600     2
5511      2
105045    2
14788     2
22649     2
15028     2
152795    2
11115     2
298721    2
13209     2
23305     2
110428    2
77221     2
42495     2
25541     2
265189    2
4912      2
69234     2
Name: count, dtype: int64


In [87]:
# Eliminar filas duplicadas basadas en la columna 'id', manteniendo la primera aparición

dfmov_sd = df3.drop_duplicates(subset='id', keep='first') #df3_sd = df3 sin duplicados

In [88]:
# Comprobar si se eliminaron duplicados
dfmov_sd[dfmov_sd.duplicated(subset=['id'], keep=False)]

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,name_coll,name_gen,production_companies,production_countries,languages,return,release_year


## creditos

#### se desanidan cast y crew

In [91]:
#Imprimir nombre de las columnas
dfc.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [92]:
dfc.shape

(45476, 3)

In [93]:
#Comprobar valores nulos
dfc.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

In [94]:
dfc.cast.iloc[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [95]:
print(dfc.cast.loc[0])

[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tN

In [96]:
# Función para extraer claves y valores
def extraer_claves_valores(diccionarios):
    if isinstance(diccionarios, str):
        try:
            diccionarios = ast.literal_eval(diccionarios)
        except (SyntaxError, ValueError):
            return []

    if not isinstance(diccionarios, list):
        return []

    resultado = []
    for diccionario in diccionarios:
        resultado.append(diccionario)
    return resultado

# Aplicar la función a la columna 'cast'
dfc['cast'] = dfc['cast'].apply(extraer_claves_valores)

In [97]:
# Convertir las listas de diccionarios en DataFrame
dfc1 = dfc['cast'].explode().apply(pd.Series)

In [98]:
dfc1[:5]

,cast_id,character,credit_id,gender,id,name,order,profile_path,0
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,NaN
0,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,NaN
0,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,NaN
0,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,NaN
0,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,NaN


In [99]:
dfc2 = dfc1[['name']]

In [100]:
dfc2[:2]

,name
0,Tom Hanks
0,Tim Allen


In [101]:
# concatenamos los valores
dfc2_ind = dfc2.groupby(dfc2.index)['name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()


In [102]:
dfc2_ind[:2]

,index,name
0,0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."


In [103]:
#Renombrar la columna 'name' por 'actors'
dfc2_ind = dfc2_ind.rename(columns = {'name' : 'actors'})


In [104]:
#Obtener el df final para actores
dfc_act = dfc2_ind[['actors']]

In [105]:
dfc_act[:2]

,actors
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."


In [106]:
# Combinar el resultado con el DataFrame original para mantener todas las filas
df_final_act = dfc[['id']].join(dfc_act, how='left')

In [107]:
df_final_act[:4]

,id,actors
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin..."


#### crew

In [109]:
# Aplicar la función a la columna 'crew'
dfc['crew'] = dfc['crew'].apply(extraer_claves_valores)

# Reemplazar listas vacías con [np.nan]
dfc['crew'] = dfc['crew'].apply(lambda x: x if x else [np.nan])

# Explosiona y convierte a DataFrame, manteniendo los NaN
dfc11 = dfc['crew'].explode().apply(pd.Series)

# Agregar un índice auxiliar único
dfc11['unique_index'] = dfc11.index

# Filtrar los datos donde 'job' es 'Director'
df_directors = dfc11[dfc11['job'] == 'Director']

In [110]:
# Agrupar y concatenar, asegurando que los NaN se mantengan
dfc22_ind = df_directors.groupby('unique_index')['name'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Combinar el resultado con el DataFrame original para mantener todas las filas
df_final_dir = dfc[['id']].join(dfc22_ind.set_index('unique_index'), how='left')

In [111]:
df_final_dir[:5]

,id,name
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


Se renombra la columna id del dataframe directores y del dataframe actores, para comprobar si sus id coinciden

In [112]:
df_final_dir = df_final_dir.rename(columns = {'id' : 'id_dir', 'name' : 'director'})

df_final_act = df_final_act.rename(columns = {'id' : 'id_act'})



In [113]:
# Se concatenan los DataFrames horizontalmente
dfcred = pd.concat([df_final_act, df_final_dir], axis=1)


In [115]:
dfcred[:2]

,id_act,actors,id_dir,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",862,John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",8844,Joe Johnston


In [116]:
# Comprobar si los valores en 'id_act' y 'id_dir' coinciden
coincidencias = dfcred['id_act'] == dfcred['id_dir']


In [117]:
coincidencias

0        True
1        True
2        True
3        True
4        True
         ... 
45471    True
45472    True
45473    True
45474    True
45475    True
Length: 45476, dtype: bool

In [118]:
# Contar el número de coincidencias
print(coincidencias.sum())

45476


Se Comprueba que los id de cada dataframe coinciden, por lo tanto se decide quedar con un solo id

In [119]:
dfcred = dfcred[['id_act', 'actors', 'director']]

In [120]:
dfcred.columns

Index(['id_act', 'actors', 'director'], dtype='object')

In [122]:
#Renombrar la columna 'id_act' por 'id_cred'
dfcred = dfcred.rename(columns = {'id_act' : 'id'})

In [123]:
dfcred[:2]

,id,actors,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston


In [124]:
dfcred.shape

(45476, 3)

#### Duplicados

In [125]:
# Contar cantidad de duplicados en la columna 'id'
dfcred['id'].duplicated(keep=False).sum()


87

In [126]:
# Contar la frecuencia de cada 'id'
frec_id = dfcred['id'].value_counts()

In [127]:
# Filtrar solo los 'id' que se repiten
id_rep = frec_id[frec_id > 1]

print("\n'id' repetidos:")
print(id_rep)


'id' repetidos:
id
141971    3
298721    2
9755      2
10991     2
99080     2
152795    2
22649     2
18440     2
5511      2
132641    2
105045    2
159849    2
187156    2
43629     2
8767      2
123634    2
157301    2
4912      2
142563    2
42495     2
11752     2
84198     2
24026     2
110428    2
24023     2
199591    2
125458    2
3057      2
116723    2
23305     2
97995     2
168538    2
12600     2
14788     2
69234     2
13209     2
15028     2
109962    2
265189    2
25541     2
119916    2
11115     2
77221     2
Name: count, dtype: int64


In [128]:
# Eliminar filas duplicadas basadas en la columna 'id', manteniendo la primera aparición

dfcred_sd = dfcred.drop_duplicates(subset='id', keep='first') #df3_sd = df3 sin duplicados

In [129]:
dfcred_sd.shape

(45432, 3)

In [130]:
#Comprobar si hay id duplicados en "credits" y en "movies"
print('Id duplicados en el dataframe movies: ', dfmov_sd[dfmov_sd.duplicated(subset = 'id', keep = False)].shape[0])
print('Id duplicados en el dataframe credicts: ', dfcred_sd[dfcred_sd.duplicated(subset = 'id', keep = False)].shape[0])

Id duplicados en el dataframe movies:  0
Id duplicados en el dataframe credicts:  0


#### Unimos los dataframes

In [131]:
dfmov_sd.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'name_coll', 'name_gen',
       'production_companies', 'production_countries', 'languages', 'return',
       'release_year'],
      dtype='object')

In [132]:
dfcred_sd.columns

Index(['id', 'actors', 'director'], dtype='object')

In [133]:
# Convertir la columna 'id' a tipo entero en ambos DataFrames
dfmov_sd['id'] = dfmov_sd['id'].astype(int)
dfcred_sd['id'] = dfcred_sd['id'].astype(int)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12792\4293325507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmov_sd['id'] = dfmov_sd['id'].astype(int)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12792\4293325507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcred_sd['id'] = dfcred_sd['id'].astype(int)


In [134]:
# Unir los DataFrames en base a la columna 'id'
df_merged = pd.merge(dfmov_sd, dfcred_sd, on='id', how='inner')



In [135]:
df_merged[:4]

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_count,name_coll,name_gen,production_companies,production_countries,languages,return,release_year,actors,director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,5415.0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English,12.451801,1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,2413.0,NaN,"Adventure,Fantasy,Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français",4.043035,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,92.0,Grumpy Old Men Collection,"Romance,Comedy","Warner Bros., Lancaster Gate",United States of America,English,0.000000,1995,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,34.0,NaN,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English,5.090760,1995,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker


#### Extraemos los datos para la Api

In [138]:
df_api = df_merged[['budget', 'revenue', 'popularity' ,'release_date', 'title', 'vote_average', 'vote_count', 'actors',
              'release_year', 'director', 'return' ]]

In [139]:
#Exportar el dataframe final para la API
df_api.to_parquet('Data_Api.parquet', index = False)

In [140]:
#Exportar el dataframe final
df_merged.to_parquet('Data_Movies.parquet', index = False)

### Data Modelo recomendacion

In [ ]:
# Se extraen las columnas necesarias para el Sistema de Recomendación
model5 = df[['title', 'name_gen', 'tagline' ,'first_actor', 'first_director']]

In [ ]:
# Se exportan los datos a formato parquet
model5.to_parquet('Data_Modelo_Recomendacion.parquet')